In [7]:
# Names list: https://el.wiktionary.org/w/index.php?title=%CE%9A%CE%B1%CF%84%CE%B7%CE%B3%CE%BF%CF%81%CE%AF%CE%B1:%CE%93%CF%85%CE%BD%CE%B1%CE%B9%CE%BA%CE%B5%CE%AF%CE%B1_%CE%BF%CE%BD%CF%8C%CE%BC%CE%B1%CF%84%CE%B1_(%CE%BD%CE%AD%CE%B1_%CE%B5%CE%BB%CE%BB%CE%B7%CE%BD%CE%B9%CE%BA%CE%AC)
words = open('greek_female_names.txt', 'r').read().splitlines()
len(words)
min(len(w) for w in words)
max(len(w) for w in words)

15

Structure Data

In [8]:
import torch
N = torch.zeros((26, 26), dtype=torch.int32)
# we need a matrix for the characters to map the counts
chars = sorted(list(set(''.join(words)))) # all chars with no duplicates
stoi = {s:i+1 for i,s in enumerate(chars)} # mapping chars to indexes
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
itos

{1: 'α',
 2: 'β',
 3: 'γ',
 4: 'δ',
 5: 'ε',
 6: 'ζ',
 7: 'η',
 8: 'θ',
 9: 'ι',
 10: 'κ',
 11: 'λ',
 12: 'μ',
 13: 'ν',
 14: 'ξ',
 15: 'ο',
 16: 'π',
 17: 'ρ',
 18: 'σ',
 19: 'τ',
 20: 'υ',
 21: 'φ',
 22: 'χ',
 23: 'ψ',
 24: 'ω',
 0: '.'}

In [9]:
print(stoi)
print(itos)

{'α': 1, 'β': 2, 'γ': 3, 'δ': 4, 'ε': 5, 'ζ': 6, 'η': 7, 'θ': 8, 'ι': 9, 'κ': 10, 'λ': 11, 'μ': 12, 'ν': 13, 'ξ': 14, 'ο': 15, 'π': 16, 'ρ': 17, 'σ': 18, 'τ': 19, 'υ': 20, 'φ': 21, 'χ': 22, 'ψ': 23, 'ω': 24, '.': 0}
{1: 'α', 2: 'β', 3: 'γ', 4: 'δ', 5: 'ε', 6: 'ζ', 7: 'η', 8: 'θ', 9: 'ι', 10: 'κ', 11: 'λ', 12: 'μ', 13: 'ν', 14: 'ξ', 15: 'ο', 16: 'π', 17: 'ρ', 18: 'σ', 19: 'τ', 20: 'υ', 21: 'φ', 22: 'χ', 23: 'ψ', 24: 'ω', 0: '.'}


Neural Network

In [10]:
# OPTIMISATION PUT TOGETHER
xs ,ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((25,25), generator=g, requires_grad=True)

number of examples:  55666


In [11]:
# gradient descent
import torch.nn.functional as F

for k in range(100):

    # forward pass
    xenc = F.one_hot(xs, num_classes=25).float()  # input to the network: one-hot encoding
    logits = xenc @ W  # predict log-counts
    counts = logits.exp()  # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True)  # probabilities for next character
    loss = -probs[torch.arange(num), ys].log().mean()
    print(loss.item())

    # backward pass
    W.grad = None # set to zero the gradient
    loss.backward()

    # update
    W.data += -50 * W.grad

3.59389591217041
3.141183376312256
2.8456952571868896
2.665436029434204
2.580273389816284
2.5274274349212646
2.487445831298828
2.4559385776519775
2.4310295581817627
2.409409284591675
2.3920626640319824
2.376012086868286
2.3633131980895996
2.35086727142334
2.341024160385132
2.330946445465088
2.323150873184204
2.3148000240325928
2.3085553646087646
2.301546335220337
2.2964844703674316
2.2905256748199463
2.2863833904266357
2.281270980834961
2.2779176235198975
2.2734832763671875
2.2707087993621826
2.266826629638672
2.264556646347046
2.261124849319458
2.259270668029785
2.256197929382324
2.254654884338379
2.2518835067749023
2.2506284713745117
2.248129367828369
2.2471039295196533
2.244802236557007
2.2439520359039307
2.241842746734619
2.24117374420166
2.2392117977142334
2.238677740097046
2.236845016479492
2.236435890197754
2.234687328338623
2.234374523162842
2.2327330112457275
2.232501745223999
2.230961561203003
2.230830669403076
2.2293648719787598
2.229276418685913
2.2278666496276855
2.2278563

In [12]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(150):
    out = []
    ix = 0
    while True:
        # ----
        # Before:
        # p = P[ix]
        # -----
        # NOW:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=25).float()
        logits = xenc @ W  # predict log-counts
        counts = logits.exp()  # counts, equivalent to N
        p = counts / counts.sum(1, keepdims=True)  # probabilities for next character
        # ------

        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

γη.
γκαντριαρσαμαρω.
καλουρικομπισαλα.
φρεικη.
πη.
ματστιουλλαχκδω.
μαμασανιαριω.
γπανναλατογιλινχανακουρυλαναιαλατεπονσερεγιαμπαυτσοπη.
τω.
βσσακιννεμανω.
ταγκρορφτιδουρατσισιαρουλα.
κη.
αναυ.
ουλιτωβεσ.
πζονου.
κγγευ.
εωρω.
ζου.
οφρωτιεννα.
σαρα.
μιλαστα.
βαπη.
κονασαρυριναφργγουλη.
αμιφρμακαντεονα.
ρασεριμπελα.
αιαγελατζξατσκιδραλη.
σσυρσαυγκατλιναλαβουναραρλδαροριταγκυλιτω.
εναννθεραλα.
λλειαια.
σαρα.
σβα.
εφιτανημυλα.
κρουλλοταδοριαλανω.
α.
πσυρια.
ζερτενιντ.
φυκεουλιτσχοφινζαναναναναλανανθρυλουλαναιωννασωναεω.
τη.
μαθειλιτστινουρουσυγινικονηρχαγουχαροζρυλαντιριναυφιανιω.
ια.
βργρρισαλιωραλλαγαναννα.
υνατσα.
φιανταινταναντσταφαχρστταραρυμαυγερετζοδα.
τα.
αλλικη.
ρυμεκξτιταραρζοπη.
ζαγδιουλαριλαλιτιαγμια.
ελανηλλη.
φχρεω.
μαπεξη.
δολανασιναμαλαλιβαλεολλειθηνθιαλευλαμαικεσναιατδελη.
κοζπιανιανα.
καυντανω.
πυρανανια.
μαμη.
βευεβδρικωτιαγκυρα.
φωα.
τικριδεβελαρω.
αριανθη.
ιανη.
κη.
διτοδαμινικαθηρελλιτιννη.
ββαλικανασω.
φιαναλαραρου.
βασικαιεβαμπρω.
σσιτρ.
αλη.
ενστελαριασουλψραπεντα.